In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDCLASS import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
shelter = AnimalShelter()
shelter.authenticate(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.query({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
#image_filename = 'my-image.png' # replace with your own image
#encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
# HTML code for the unique identifier
unique_identifier = html.Div(
    children="This is my unique identifier: Ubaidahallah Kramez",
    style={'font-style': 'italic', 'font-size': '12px', 'margin-top': '10px'}  
)
image_filename = '/home/ubaidahallahk_snhu/Module4/Grazioso Salvare Logo.png'

encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode() # Encode image for inline display



# HTML code for the logo

logo_image = html.Img(

    src='data:image/png;base64,{}'.format(encoded_image),
    
    style={'height': '100px', 'width': 'auto', 'margin-bottom': '10px'} # Adjust styling as needed
    
)
app.layout = html.Div([
    logo_image,
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(children=[unique_identifier]),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
      [
          dcc.RadioItems(
              id="filter-type",
              options=[
                  {"label": "Water Rescue", "value": "water_rescue"},
                  {"label": "Mountain Rescue", "value": "mountain_rescue"},
                  {"label": "Disaster Rescue", "value": "disaster_rescue"},
                  {"label": "Reset", "value": "all"},  # Reset option
              ],
              value="all",  # Default value to show all initially
          )
      ],
      style={"display": "flex", "justify-content": "center", "margin-bottom": "10px"},
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                        # **1. Enable Sorting**
        sort_action='native',
        
        # **2. Enable Filtering**
        filter_action='native',
        
        # **3. Enable Pagination**
        page_action='native',
        page_current=0,
        page_size=10,
        
        # **4. Row Selection**
        row_selectable='single',  # Can be 'single' or 'multi'
        selected_rows=[],
        
        # **5. Column Resizing**
        style_table={'overflowX': 'auto'},  # Allows horizontal scrolling for wider tables
        editable=True,  # Make cells editable if needed
        
        # **6. Conditional Formatting**
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},  # Alternate row shading
                'background_color': 'rgb(248, 248, 248)'
            },
            {
                'if': {'column_id': 'some_column'},  # Highlight a specific column
                'background_color': '#D2F3FF',
                'fontWeight': 'bold'
            }
        ]            
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
                 id='graph-id',
                 className='col s12 m6',
                 children=[dcc.Graph(id='pie-chart')]  # Add the dcc.Graph component here
             ),
             html.Div(
                 id='map-id',
                 className='col s12 m6',
             )
         ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

    if filter_type == "all":
        df = pd.DataFrame.from_records(shelter.query({}))  # Reload all data
    elif filter_type == "water_rescue":
        db_results = shelter.query_water_rescue()
        df = pd.DataFrame.from_records(db_results)
    elif filter_type == "mountain_rescue":
        db_results = shelter.query_mountain_rescue()
        df = pd.DataFrame.from_records(db_results)
    elif filter_type == "disaster_rescue":
        db_results = shelter.query_disaster_rescue()
        df = pd.DataFrame.from_records(db_results)
    else:
        return [] #Return empty if there is an error

    if not df.empty:
        df.drop(columns=['_id'], inplace=True, errors='ignore')
        return df.to_dict('records')
    else:
        return [] #Return empty if there is an error

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return dash.no_update

    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty: #Check if the dataframe is empty
        return dcc.Graph(figure=px.pie(pd.DataFrame({'No Data': [1]}), names=pd.Series(['No Data']), values=pd.Series([1]), title='No Data Available'))

    # Dynamic labels for the pie chart
    breed_counts = dff['breed'].value_counts()
    labels = breed_counts.index.tolist()
    values = breed_counts.values.tolist()

    fig = px.pie(dff, names=labels, values=values, title='Animal Breeds') #Use the dynamic labels and values
    return dcc.Graph(figure=fig)
    

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if not viewData or not index:  # Handle case where there's no data or no selection
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id")
                ])
        ]
    
    # Convert viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0  # Use the first selected row or default to 0

    # Example with lat/lon indices - adjust these to match your data
    latitude = dff.iloc[row, 13]  # Replace 13 with the actual latitude column index
    longitude = dff.iloc[row, 14]  # Replace 14 with the actual longitude column index
    tooltip = dff.iloc[row, 4]  # Replace 4 with the tooltip column index
    popup_text = dff.iloc[row, 9]  # Replace 9 with the popup text column index

    # Return the updated map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[latitude, longitude], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[latitude, longitude],
                          children=[
                              dl.Tooltip(tooltip),
                              dl.Popup([html.H1("Animal Name"), html.P(popup_text)])
                          ])
            ])
    ]


app.run_server(debug=True)


MongoDB connection established. You can now proceed with CRUD operations.
Login successful! You are connected to MongoDB.
Dash app running on http://127.0.0.1:10653/
